In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

In [2]:
model = load_model('handEmo.h5')

In [3]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [6]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
9 0.44441026
9 0.45208767
9 0.4076508
11 0.34963903
11 0.30146036
11 0.36558953
11 0.35345516
11 0.35006046
9 0.41659957
9 0.9454015
8 0.7320499
8 0.8492048
8 0.85677093
8 0.75161046
11 0.62857044
5 0.9997727
5 0.9739446
5 0.8734677
5 0.9202446
5 0.99058354
5 0.9861436
5 0.9812207
5 0.98773193
5 0.9948643
5 0.9961739
5 0.9962459
5 0.99572057
5 0.9982102
5 0.9956267
5 0.98642856
11 0.55458844
11 0.85662675
1 0.9237154
8 0.8775978
8 0.94186497
8 0.8736517
8 0.93143594
8 0.95607555
8 0.9416184
8 0.93891966
8 0.91922677
8 0.895213
8 0.9366924
8 0.5188187
8 0.7335502
3 0.33428648
3 0.72407985
11 0.62048215
8 0.6013374
2 0.4703098
5 0.71676093
5 0.8099428
7 0.62647843
7 0.62620896
5 0.66065806
5 0.7624884
5 0.84584963
5 0.7617589
5 0.86141384
5 0.9080477
5 0.931877
5 0.89771473
5 0.9735266
11 0.47491318
2 0.18644145
5 0.9916334
5 0.98974425
5 0.9975344
5 0.99438536
5 0.9955011
5 0.9965418
5 0.99382776
5 0.9970112
5 0.99568355
5 0.9975051
5 0.99562114
5 0.98500776

11 0.35722148
11 0.35394457
4 0.39957693
4 0.4223076
11 0.3325658
11 0.2759358
3 0.25114605
11 0.43539906
3 0.27787998
3 0.33995828
3 0.19315663
11 0.36273167
11 0.2996178
3 0.22757322
11 0.47960845
11 0.3449185
11 0.46962044
11 0.5245326
11 0.62684053
11 0.5416683
3 0.30284837
11 0.4427955
11 0.2844126
11 0.49319628
11 0.42066017
11 0.48165613
11 0.4680587
11 0.50272477
11 0.5988938
11 0.52586716
11 0.48641095
11 0.65409493
11 0.6700056
11 0.64709777
11 0.57665384
11 0.35486582
11 0.53468895
11 0.44271055
11 0.65863895
11 0.5533479
11 0.49742708
11 0.6455229
11 0.5372639
11 0.49807754
11 0.4817121
11 0.616173
11 0.61835897
11 0.62163514
11 0.56559837
11 0.5263443
11 0.5287082
11 0.58376586
11 0.5364134
11 0.6263426
11 0.43150485
11 0.4971281
11 0.62846535
11 0.524497
11 0.56038594
11 0.63594604
11 0.42253628
11 0.44249073
11 0.466439
11 0.5715512
11 0.5124422
11 0.5164588
11 0.48265836
8 0.398301
11 0.5597306
11 0.374886
11 0.41531405
11 0.53319377
11 0.47410613
11 0.6492135
11 0.4874

9 0.98377997
9 0.98229146
9 0.74477047
9 0.740975
9 0.7791919
9 0.9179458
6 0.37885392
9 0.7890665
6 0.48822904
9 0.57382005
9 0.7560794
9 0.48017576
9 0.7550058
6 0.5271708
9 0.71149975
9 0.62669617
9 0.73254937
9 0.88037646
9 0.867668
9 0.72514826
9 0.88369685
9 0.95127666
9 0.91287506
9 0.7081159
6 0.4399651
9 0.6719899
9 0.8693977
9 0.66345704
9 0.75838536
9 0.44181788
9 0.86697453
6 0.5806164
9 0.6125231
9 0.7908962
6 0.55837905
6 0.48475346
9 0.99133533
9 0.9862726
9 0.94399416
9 0.94121414
9 0.77292335
9 0.732387
9 0.63552207
9 0.8876221
9 0.7581542
9 0.61707115
9 0.852436
9 0.6592777
10 0.9377104
10 0.8491588
10 0.542678
10 0.5019233
10 0.7430946
10 0.51092017
9 0.95783204
9 0.95969033
9 0.987723
9 0.9721752
9 0.9932852
9 0.9648899
9 0.94657
9 0.79387194
9 0.57410073
9 0.6467281
10 0.97820204
10 0.9062504
9 0.4418574
10 0.76894104
10 0.79614043
10 0.8472846
10 0.816967
10 0.5937243
10 0.8916935
10 0.84943223
6 0.4590609
10 0.37131283
10 0.35417134
10 0.3026635
10 0.8241111
10 0

KeyboardInterrupt: 